In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import math
from implementations import *
from proj1_helpers import *
from misc_helpers import *
from plot_functions import *
from ml_math import *
%load_ext autoreload
%autoreload 2

# Optimizing Learning by "brute force"

### Load Data

In [2]:
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y_data, x_data, ids = load_csv_data(DATA_TRAIN_PATH)

### Standardize

In [4]:
y = (y_data+1)/2
x = normalize(x_data)

## Find the best number of gamma

### Define useful functions

In [5]:
# 'New' build_poly used in the labs correction, the old one should also work but you're never sure
def build_poly(x, degree):
    """
    ----------------------------------------------------------------------------
    Function that builds a matrix x to a polynomial matrix of of powers of x.
    ----------------------------------------------------------------------------
    Input:
    - x         parameter, (nsamples, nfeature) np.array
    - degree    number of dimension, if it is a float number, convert it into
                an integer
    Output:
    - poly      polynomial matrix, (nsamples, (nfeature*degree) + 1), np.array
    ----------------------------------------------------------------------------
    """
    poly = np.ones((len(x), 1))
    for deg in range(1, int(degree+1)):
        poly = np.c_[poly, np.power(x, deg)]
    return poly

# Build_k_indices to do cross validation
def build_k_indices(y, k_fold, seed):
    """
    ----------------------------------------------------------------------------
    Function that builds "k_fold" sets of randomly distributed indices for an 
    array of size y.shape[0]
    ----------------------------------------------------------------------------
    Input:
    - y         array to be permuted in the future, (nsamples, 1) np.array
    - k_fold    number of sets of same size to be build, integer
    - seed      seed for the random number generator
    Output:
    - answ      k_folds set of randomly distributed indices, (k_fold, 
                nsamples/k_fold), np.array
    ----------------------------------------------------------------------------
    """
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval : (k + 1) * interval] for k in range(k_fold)]
    return np.array(k_indices)


def cross_validation_visualization(param, tested_parameter, rmse_tr, rmse_te, 
                                   graph_type = "log", label="", color1 = "r", color2 = "b"):
    """
    ----------------------------------------------------------------------------
    Function that plots the error in training and in test as a function of a
    given parameter, then saves it with the name "cross_validation".
    ----------------------------------------------------------------------------
    Input:
    - param             values of the parameter, array
    - tested_parameter  name of the parameter, string
    - rmse_tr           training error, array
    - rmse_te           test error, array
    - graph_type        choose between a loglog graph - "log" - and a semilogy
                        - otherwise - graph, string, (Default = "log")
    - label             gives additional information on the label, string
                        (Default = "")
    - color1            color of the train data (Default = "r")
    - color2            color of the test data (Default = "b")
    Output:
    ----------------------------------------------------------------------------
    """
    if (graph_type == "log"):
        plt.loglog(param, rmse_tr, marker=".", color=color1, label='train error' + " " + label)
        plt.loglog(param, rmse_te, marker=".", color=color2, label='test error' + " " + label)
    else:
        plt.semilogy(param, rmse_tr, marker=".", color=color1, label='train error' + " " + label)
        plt.semilogy(param, rmse_te, marker=".", color=color2, label='test error' + " " + label)
    plt.xlabel(tested_parameter)
    plt.ylabel("error")
    plt.title("cross validation")
    plt.legend(loc=2)
    plt.grid(True)
    plt.savefig("cross_validation_" + tested_parameter)

### Implement cross-validation

In [6]:
def cross_validation(y, x, k_indices, k, degree, gamma, lambda_, max_iters, method, w0):
    """
    ----------------------------------------------------------------------------
    Function that implements the cross-validation given multiple parameters.
    Returns the calculated, loss in both the training and test sets, and the
    weights
    ----------------------------------------------------------------------------
    Input:
    - y         "measured" objective function, (nsamples,1) np.array
    - x         features, (nsamples,nfeatures) np.array
    - k_indices k_fold sets of randomly distributed indices, (k_fold, 
                nsamples/k_fold), np.array
    - k         number of the set to use as test set
    - degree    number of dimension
    - gamma     step size, scalar in ]0,1[
    - lambda_   regularization parameter, scalar>0
    - max_iters # of iterations after which the procedure will stop, int>0
    - method    method to use in this cross-validation iteration, integer 
                between 0 to 6 (0 = least_square_GD, 1 = least_square_SGD,
                2 = least_squares, 3 = ridge_regression, 4 = 
                logistic_regression, 5 = reg_logistic_regression, 6 = 
                new_ridge_regression_GD)
    - w0
    Output:
    - w             obtained weights, (nfeatures,1) np.array
    - loss_tr       loss on the training set, scalar
    - loss_te       loss on the test set, scalar
    ----------------------------------------------------------------------------
    """
    # get k'th subgroup in test, others in train
    te_indice = k_indices[k]
    tr_indice = k_indices[~(np.arange(k_indices.shape[0]) == k)]
    tr_indice = tr_indice.reshape(-1)
    y_te = y[te_indice]
    y_tr = y[tr_indice]
    x_te = x[te_indice]
    x_tr = x[tr_indice]
    loss_function = "MSE"
    
    if method == 0:
        tx_tr = build_poly(x_tr, degree)
        tx_te = build_poly(x_te, degree)
        initial_w = np.ones(len(tx_tr[1,:])) * w0
        w, loss_tr = least_squares_GD(y_tr, tx_tr, initial_w, max_iters, gamma, verbose=False)
    if method == 1:
        tx_tr = build_poly(x_tr, degree)
        tx_te = build_poly(x_te, degree)
        initial_w = np.ones(len(tx_tr[1,:])) * w0
        w, loss_tr = least_squares_SGD(y_tr, tx_tr, initial_w, 1, max_iters, gamma, verbose=False)
    if method == 2:
        tx_tr = build_poly(x_tr, degree)
        tx_te = build_poly(x_te, degree)
        w, loss_tr = least_squares(y_tr, tx_tr)
    if method == 3:
        tx_tr = build_poly(x_tr, degree)
        tx_te = build_poly(x_te, degree)
        w, loss_tr = ridge_regression(y_tr, tx_tr, lambda_)
    if method == 4:
        loss_function = "MSE"
        tx_tr = build_poly(x_tr, degree)
        tx_te = build_poly(x_te, degree)
        initial_w = np.ones(len(tx_tr[1,:])) * w0
        w, loss_tr = logistic_regression(y_tr, tx_tr, initial_w, max_iters, gamma, verbose = False, use_SGD = True, batch_size = 1, loss_function = "LL2")
    if method == 5:
        loss_function = "MSE"
        tx_tr = build_poly(x_tr, degree)
        tx_te = build_poly(x_te, degree)
        initial_w = np.ones(len(tx_tr[1,:])) * w0
        w, loss_tr = reg_logistic_regression(y_tr, tx_tr, lambda_, initial_w, max_iters, gamma, verbose = False, use_SGD = True, batch_size = 1, loss_function = "LL2")
    if method == 6:
        tx_tr = build_poly(x_tr, degree)
        tx_te = build_poly(x_te, degree)
        initial_w = np.ones(len(tx_tr[1,:])) * w0
        w, loss_tr = new_ridge_regression_GD(y_tr, tx_tr, initial_w, lambda_, gamma, max_iters, verbose=False)
    
    # calculate the loss for test data
    loss_te = compute_loss(y_te, tx_te, w, loss_function)
    return loss_tr, loss_te, w

### Define tests on all parameters

In [7]:
def test_all():
    """
    ----------------------------------------------------------------------------
    Function that test all the variable parameters in a huge for loop.
    Warning! Takes a really long time to compute.
    ----------------------------------------------------------------------------
    """
    
    
    # define the system constant parameters
    seed = 1
    k_fold = 4
    k_indices = build_k_indices(y, k_fold, seed)
    max_iters = 20
    rmse_tr = []
    rmse_te = []
    best_rmse_te = np.ones(7)*999999
    best_rmse_tr = np.ones(7)
    best_method = np.ones(7)
    best_degree = np.ones(7)
    best_gamma = np.ones(7)
    best_lambda = np.ones(7)
    best_w0 = np.ones(7)
    i = 0
    
    # define the system variable parameters
    methods = np.linspace(0, 6, 7)
    degrees = np.linspace(3, 7, 5)
    gammas = np.logspace(-3, 0, 4)
    lambdas = np.logspace(-3, 1, 5)
    w0s = np.linspace(-5, 5, 3)
    
    for method in methods:
        for degree in degrees:
            for gamma in gammas:
                for lambda_ in lambdas:
                    for w0 in w0s:
                        rmse_tr_tmp = []
                        rmse_te_tmp = []
                        for k in range(k_fold):
                            loss_tr, loss_te, _ = cross_validation(y, x, k_indices, k, degree, gamma, lambda_, max_iters, method, w0)
                            rmse_tr_tmp.append(loss_tr)
                            rmse_te_tmp.append(loss_te)
                        rmse_tr = np.mean(rmse_tr_tmp)
                        rmse_te = np.mean(rmse_te_tmp)
                        if rmse_te < best_rmse_te[int(method)]:
                            best_rmse_te[int(method)] = rmse_te
                            best_rmse_tr[int(method)] = rmse_tr
                            best_method[int(method)] = method
                            best_degree[int(method)] = degree
                            best_gamma[int(method)] = gamma
                            best_lambda[int(method)] = lambda_
                            best_w0[int(method)] = w0
                        i = i + 1
                        print(str(i / (7 * 5 * 4 * 5 * 3) * 100) + "% done")
    # print("Best rmse_te = %s ; Best rmse_tr = %s ; Best method = %s ; Best degree = %s ; Best gamma = %s ; Best lambda = %s ; Best w0 = %s" % (best_rmse_te, best_rmse_tr, best_method, best_degree, best_gamma, best_lambda, best_w0))
    print(best_rmse_te)
    print(best_rmse_tr)
    print(best_method)
    print(best_degree)
    print(best_gamma)
    print(best_lambda)
    print(best_w0)
    return best_rmse_te, best_rmse_tr, best_method, best_degree, best_gamma, best_lambda, best_w0

In [8]:
test_all()

0.047619047619047616% done
0.09523809523809523% done
0.14285714285714285% done
0.19047619047619047% done
0.2380952380952381% done
0.2857142857142857% done
0.33333333333333337% done
0.38095238095238093% done
0.4285714285714286% done
0.4761904761904762% done
0.5238095238095238% done
0.5714285714285714% done
0.6190476190476191% done
0.6666666666666667% done
0.7142857142857143% done
0.7619047619047619% done
0.8095238095238094% done
0.8571428571428572% done
0.9047619047619048% done
0.9523809523809524% done
1.0% done
1.0476190476190477% done
1.0952380952380953% done
1.1428571428571428% done
1.1904761904761905% done
1.2380952380952381% done
1.2857142857142856% done
1.3333333333333335% done
1.380952380952381% done
1.4285714285714286% done
1.4761904761904763% done
1.5238095238095237% done
1.5714285714285716% done
1.6190476190476188% done
1.6666666666666667% done
1.7142857142857144% done
1.7619047619047619% done
1.8095238095238095% done
1.8571428571428572% done
1.9047619047619049% done
1.9523809

KeyboardInterrupt: 

In [ ]:
def test_ridge_regression():
    """
    ----------------------------------------------------------------------------
    Function that try to find the best parameters for ridge regression.
    ----------------------------------------------------------------------------
    """
    
    
    # define the system constant parameters
    seed = 1
    k_fold = 4
    k_indices = build_k_indices(y, k_fold, seed)
    max_iters = 20
    rmse_tr = []
    rmse_te = []
    best_rmse_te = 999999
    best_rmse_tr = 0
    best_degree = 0
    best_lambda = 0
    i = 0
    j = 0
    
    gamma = 0
    method = 3
    w0 = 0
    
    # define the system variable parameters
    degrees = np.linspace(3, 4, 2)
    lambdas = np.logspace(-6, 0, 20)
    
    for degree in degrees:
        for lambda_ in lambdas:
            rmse_tr_tmp = []
            rmse_te_tmp = []
            for k in range(k_fold):
                loss_tr, loss_te, _ = cross_validation(y, x, k_indices, k, degree, gamma, lambda_, max_iters, method, w0)
                rmse_tr_tmp.append(loss_tr)
                rmse_te_tmp.append(loss_te)
            rmse_tr.append(np.mean(rmse_tr_tmp))
            rmse_te.append(np.mean(rmse_te_tmp))
            if rmse_te[i] < best_rmse_te:
                best_rmse_te = rmse_te[i]
                best_rmse_tr = rmse_tr[i]
                best_degree = degree
                best_lambda = lambda_
            i = i + 1
            print(str(i / (2 * 20) * 100) + "% done")
        cross_validation_visualization(lambdas, "lambda", rmse_tr[int((j)*20):int((j+1)*20)], rmse_te[int((j)*20):int((j+1)*20)], "log","degree = " + str(degree),([0, 0, j]),([1, 0, j]))     
        j = j + 1 
    print(best_rmse_te)
    print(best_rmse_tr)
    print(best_degree)
    print(best_lambda)
    return best_rmse_te, best_rmse_tr, best_degree, best_lambda

In [ ]:
test_ridge_regression()